In [14]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
import sklearn
from math import *
from prettytable import PrettyTable
from functools import partial
from decimal import Decimal
import sympy

plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]
plt.rcParams["axes.unicode_minus"] = False  # 解决无法显示符号的问题
palette = "deep"
sns.set(font="Microsoft YaHei", font_scale=1.2, palette=palette)  # 解决Seaborn中文显示问题

In [15]:
def nm2m(海里):
    return 海里 * 1852

In [114]:
def getFns(
    起始点: tuple, 南北长度: float, 东西长度: float, 换能器开角: float, 坡度: float, 海域中心点水深: float
):
    """

    所有角度入参都是弧度制！！！函数内部不做转换

    Arguments:

    起始点 - 单位米，例如：(0, 0)

    南北长度 - 单位米

    东西长度 - 单位米

    换能器开角 - 单位弧度制

    坡度 - 单位弧度制

    Returns:
    """
    起始点x, 起始点y = 起始点

    alpha = 坡度
    theta = 换能器开角

    """
    北^
    A--------------------B----------> y方向
    |                    |
    |                    |
    |                    |
    D--------------------C
    |
    |
    x方向
    """
    AD = 南北长度
    AB = 东西长度

    p待测区A = sympy.Point2D(0, 0)
    p待测区B = sympy.Point2D(0, AB)
    p待测区C = sympy.Point2D(AD, AB)
    p待测区D = sympy.Point2D(AD, 0)

    def 根据参考点计算相对点的海深度(参考点深度, 测量船距离参考点, 测线方向夹角):
        beta = 测线方向夹角
        d = 测量船距离参考点
        d0 = 参考点深度
        val = d0 + (d * (cos(beta) * sin(alpha))) / sqrt(
            1 - cos(beta) ** 2 * sin(alpha) ** 2
        )
        return val

    def 计算覆盖宽度(测线方向夹角, 测量船距离参考点, 参考点深度):
        def gamma(beta):
            val = asin(abs(sin(alpha * sin(beta))))
            return val

        gammar = gamma(测线方向夹角)
        deep = 根据参考点计算相对点的海深度(参考点深度, 测量船距离参考点, 测线方向夹角)
        val = (
            deep
            * cos(gammar)
            * sin(theta)
            / (cos(gammar - theta / 2) * cos(gammar + theta / 2))
        )
        return val

    def 计算测线长度(测线方向夹角):
        beta = 测线方向夹角
        betaQuote = atan(东西长度 / 南北长度)
        if 测线方向夹角 < betaQuote:
            return 南北长度 / cos(beta)
        else:
            return 东西长度 / cos(pi / 2 - beta)

    def 直线转为测线线段(测线直线: sympy.Ray2D) -> sympy.Segment:
        source = 测线直线.source
        source3d = sympy.Point(source.x, source.y, 0)
        lineAB = sympy.Segment2D(p待测区A, p待测区B)
        lineDC = sympy.Segment2D(p待测区D, p待测区C)
        lineAD = sympy.Segment2D(p待测区A, p待测区D)
        lineBC = sympy.Segment2D(p待测区B, p待测区C)

        for i in [lineAB, lineAD, lineBC, lineDC]:
            pIntersection = 测线直线.intersection(i)
            pInter3d = sympy.Point(pIntersection.x, pIntersection.y, 0)
            if pIntersection:
                return sympy.Segment(source3d, pInter3d)

    def 计算重叠率():
        pass

    def get各个参考平面():
        res = dict()
        res["x轴向量"] = sympy.Point(1, 0, 0)
        res["y轴向量"] = sympy.Point(0, 1, 0)
        res["z轴向量"] = sympy.Point(0, 0, 1)
        res["xoy面"] = sympy.Plane((1, 0, 0), normal_vector=(0, 0, 1))
        res["xoz面"] = sympy.Plane((1, 0, 0), normal_vector=(0, 1, 0))

        lineAC = sympy.Segment2D(p待测区A, p待测区C)
        res["海域中心点2d"] = lineAC.midpoint
        res["海域中心点3d"] = sympy.Point3D(
            res["海域中心点2d"].x,
            res["海域中心点2d"].y,
            0 - 海域中心点水深,
        )

        res["坡面AB向量"] = sympy.Point(-cos(alpha), 0, sin(alpha))
        res["坡面"] = sympy.Plane(
            res["海域中心点3d"],
            res["海域中心点3d"] + res["y轴向量"] * 1,
            res["海域中心点3d"] + res["坡面AB向量"] * 1,
        )

        return res

    return 计算覆盖宽度, 计算测线长度, 计算重叠率, 直线转为测线线段, get各个参考平面


计算覆盖宽度, 计算测线长度, 计算重叠率, 直线转为测线线段, get各个参考平面 = getFns(
    起始点=(nm2m(0), nm2m(0)),
    南北长度=nm2m(2),
    东西长度=nm2m(4),
    换能器开角=radians(120),
    坡度=radians(1.5),
    海域中心点水深=120,
)


def 目标函数(测线方向夹角, 采样率=10):
    """
    Arguments:

    测线方向夹角 - 弧度制

    采样率 - 单位米

    Returns:

    测线长度
    """

    beta = 测线方向夹角
    首条测线出发点 = sympy.Point(0, 0, 0)

    """
    假设所有测线方案，第一条测线都要覆盖坐标原点
    那么此时判断第一条测线的方位就很重要
    只要第一条测线的方程确定了，后面的可以紧接着第一条测线继续计算
    """

    # 计算出第一条测线方程
    首条测线斜率 = tan(pi - beta)
    首条测线方向向量2d = sympy.Line2D(首条测线出发点, slope=首条测线斜率).direction.unit
    首条测线射线2d = sympy.Ray2D(首条测线出发点, 首条测线方向向量2d)
    首条测线 = 直线转为测线线段(首条测线射线2d)

    assert isinstance(首条测线, sympy.Segment)

    # 判断第一条测线在海底的覆盖区域是否包含 海底边角的极端点
    def 走线(测线: sympy.Segment, 步进: float):
        pSource = 测线.source

        for delta in np.arange(0, 测线.length + 步进, 步进):
            前进后的点 = pSource + 测线.direction.unit * delta
            pass

    # 如果不是，重新计算第一条测线方程

    计算测线长度(测线方向夹角)

In [115]:
test = get各个参考平面()
test

{'x轴向量': Point3D(1, 0, 0),
 'y轴向量': Point3D(0, 1, 0),
 'z轴向量': Point3D(0, 0, 1),
 'xoy面': Plane(Point3D(1, 0, 0), (0, 0, 1)),
 'xoz面': Plane(Point3D(1, 0, 0), (0, 1, 0)),
 '海域中心点2d': Point2D(1852, 3704),
 '海域中心点3d': Point3D(1852, 3704, -120),
 '坡面AB向量': Point3D(-999657324975557/1000000000000000, 0, 261769483078731/10000000000000000),
 '坡面': Plane(Point3D(1852, 3704, -120), (261769483078731/10000000000000000, 0, 999657324975557/1000000000000000))}

In [1]:
from mcm_cn2023.utils import util

util.计算两平面夹角(test["坡面"], test["xoy面"], True)

NameError: name 'test' is not defined